# Prepare corpus 
clean strings and add corrected to see if the predictions improve

In [1]:
# handle  csv read and write
import lib.handle_csv as csvh
# library for connecting to the db
# managing files and file paths
from pathlib import Path

import enchant
#d = enchant.Dict("en")

#d.check("Hello")

## Handle non dictionary instances
if a word is not in the dictionary it can either:
  - Merge of two or more words (space by PDF reader)
  - A non-dictionary word (Proper name, abreviation)
  - A nom-word (code word, Acronym Chemical formula, reference, number)

In [2]:
from IPython.display import clear_output

# split difficult word
def split_difficult(a_word,uw):
    # try to split with previously found words
    collected = split_with_used(a_word,uw)
    # split words manually
    if len(collected) == 0:
        collected = split_manually(a_word)
    return collected

# try to split with previously found words
#   try to match existing words
def split_with_used(str_this, used_words):
    split_word = []
    while str_this != "":
        temp = ""
        for auw in used_words:
            if auw in str_this and str_this.index(auw) == 0:
                if len(auw) > len(temp):
                    temp = auw
        if temp != "":
            split_word.append(temp)
            str_this=str_this.replace(temp, "")
        else:
            break
    if str_this != "" and split_word != []:
        split_word.append(str_this)
    return split_word

# split words manually
def split_manually(a_word):
    collected = []
    while a_word!= "":
        addthis, a_word = get_from_string(a_word)
        collected.append(addthis)
    return collected

# get one word and remainder of string
def get_from_string(string_value):
    element_value = ""
    print("************ get *************************")
    print("split and add this: ", string_value)
    element_value = extract_custom(string_value)
    string_value = string_value.replace(element_value,'')
    string_value = remove_extra_commas(string_value)
    return element_value, string_value

# remove punctuation on the edges
def remove_extra_commas(str_this):
    str_this = str_this.replace(", ,", "")
    str_this = str_this.replace("; ;", "")
    str_this = str_this.replace(" ;", ";")
    str_this = str_this.strip()
    if len(str_this) == 0:
        return ""
    return remove_lt_punctuation(str_this)

# remove leading and trailing punctuation from string
def remove_lt_punctuation(str_this):
    if str_this[0] in (",",";",".","(","{","[",")","}","]") : str_this = str_this[1:] 
    if str_this[-1:] in (",",";",".","(","{","[",")","}","]") : str_this = str_this[:-1] 
    return str_this.strip()

# select portion of string to break
def extract_custom(split_this):
    print(split_this)
    decimal_str = ""
    for indx in range(0, len(split_this)):
        val = " "
        if (indx % 10 == 0): val = str(int(indx/10)) 
        decimal_str += val
    print(decimal_str)
    unit_str = ""
    for indx in range(0, len(split_this)):
        val = str(indx%10)
        if (indx % 10 == 0): val = "0" 
        unit_str += val
    print(unit_str)
    print("start")
    str_start = int(input())
    print("end")
    str_end = int(input())
    clear_output()
    return split_this[str_start:str_end+1]

# if word is dictionary word do not check and larger than 3
def need_to_check(a_word):
    check_it = False
    #if string is alphanumeric
    only_alphanum = re.sub(r'[^a-zA-Z0-9]', '', a_word)
    only_alpha = re.sub(r'[^a-zA-Z]', '', a_word)
    only_num = re.sub(r'[^0-9]', '', a_word) 
    if len(only_alpha) >= 3 and len(only_alpha) > len(only_num):
        check_it = True
    return check_it

# return only alphanumeric characters in word
def get_alphanum(a_word):
    ret_str = re.sub(r'\W+', '', a_word)
    return ret_str

def add_word_to_list(words_list, a_word):
    if not a_word in words_list:
        words_list.append(a_word)
    return words_list

def check_non_dict(a_word, words_list):
    return (a_word in words_list)

def check_difficult(a_word, non_dict, used_words):
    print("D WORD:", a_word)
    opt_user=""
    diff_string = []
    if not need_to_check(a_word):
        opt_user="c"
    # try to split with used 
    diff_string = split_with_used(a_word, used_words)
    if diff_string !=[]:
        opt_user="a"
    while not opt_user in ['a','b','c']:
        print("options:\n\ta) split and add\n\tb) add to non dictionary\n\tc) ignore")
        opt_user = input()
    if opt_user == 'a':
        print("need to split this")
        diff_string = split_difficult(a_word, used_words)
    elif opt_user == 'b':
        print("add to non dictionary")
        a_word = re.sub(r'\W+', '', a_word)
        non_dict = add_word_to_list(non_dict, a_word)
        diff_string = [a_word]
    elif opt_user == 'c':
        print("ignore")
        diff_string = [a_word]
    return diff_string


### Get data and saved lists

In [3]:
import re

def revTrainTestData(pdf_data_search_dir, in_name):
    data_mentions, dm_headers = csvh.get_csv_data(Path(pdf_data_search_dir, in_name+'.csv'))    
    return data_mentions, dm_headers

def save_list (a_list, name):
    with open(name, "w") as f:
        for s in a_list:
            try: 
                f.write(str(s) +"\n")
            except: 
                print(" could not write", s )

def get_list(name):
    a_list = []
    with open(name, "r") as f:
        for line in f:
            a_list.append(line.strip())
    return a_list

# Correct the description sentence
#  get the sentences
#  check if they are in the dictionary or on the non_dict words
#    - if OK add to the return sentence list
#    - else check the difficult work to see if it needs correcting
#        add whathever is returned to the sentece list
#  join the words in the list to form the corrected sentence
def correct_desc(a_sentence, used_words=[], non_dict=[]):
    eng_dict = enchant.Dict("en")
    return_sentence = []
    items = a_sentence.split()
    for itm_idx in items: 
        check_word = get_alphanum(itm_idx)
        if check_word != "" and (eng_dict.check (check_word) or check_non_dict(check_word, non_dict)):
            return_sentence.append(check_word)
            used_words = add_word_to_list(used_words, check_word)
        else: 
            diff_list = check_difficult(itm_idx, non_dict, used_words)
            for aw in diff_list:
                return_sentence.append(aw)
                used_words.append(aw)
    return " ".join(return_sentence), used_words

In [7]:
# get train and test data

# working directory
pdf_data_search_dir = "./data_search_pdf_b"

# file containing train and test data
train_test_file =  'test_train01_rev'
# non dictionary words
non_dict_file = "non_dict_list.txt"
# used_word file
used_dict_file = "used_dict_list.txt"
# results file
out_test_file =  'test_train01_rev'

data, headers = revTrainTestData(pdf_data_search_dir, train_test_file)
uw=get_list(Path(pdf_data_search_dir, used_dict_file))
nd= get_list(Path(pdf_data_search_dir, non_dict_file))

#print(data)
for item in data:
    if(data[item]["corrected"] == ""):
        entry_string = data[item]["desc"]
        print("Original", data[item]["desc"])
        result_string , uw = correct_desc(data[item]["desc"],uw,nd)
        clear_output()
        data[item]["corrected"] = result_string
        print("Original", data[item]["desc"])
        print("Corrected", result_string )
        print("continue?")
        opt_user = input()
        if opt_user.lower() == "n":
            save_list(nd, Path(pdf_data_search_dir, non_dict_file))
            csvh.write_csv_data(data, Path(pdf_data_search_dir,  out_test_file +'.csv'))
            if len(uw) > 0:
                uw = set(uw)
                uw = list(uw)
                uw.sort()
                save_list(uw, Path(pdf_data_search_dir, used_dict_file))
            break

Original acquired concurrently with the copper reference foil placed between It and Iref XASdata processing, and EXAFS analysis were performed using IFEFFIT18 with theHorae package19 (Athena and Artemis).
Corrected acquired concurrently with the copper reference foil placed between It and Iref XAS data processing and EXAFS analysis were performed using IFEFFIT 1 8 with the Horae package 1 9 Athena and Artemis
continue?
n
 could not write ï¬nalizing
 could not write ï¬ndingsof


In [11]:
            save_list(nd, Path(pdf_data_search_dir, non_dict_file))
            csvh.write_csv_data(data, Path(pdf_data_search_dir,  out_test_file +'.csv'))
            if len(uw) > 0:
                uw = set(uw)
                uw = list(uw)
                uw.sort()
                save_list(uw, Path(pdf_data_search_dir, used_dict_file))
